<a href="https://colab.research.google.com/github/Hamouda-Yasmine/PFE-X-ray_image_search_using_CBIR_deepL/blob/main/Siamese%20Network%20CBIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jun 22 17:08:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
from itertools import chain
import os
import random
import zipfile
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import torch
import torchvision.transforms as T
import shutil

In [ ]:
class TripletData(Dataset):
    def __init__(self, path, transforms, split):
        self.path = path
        self.split = split    # train or valid
        self.cats = 58      # number of categories
        self.transforms = transforms

    def __getitem__(self, idx):
        # our positive class for the triplet
        idx = str(idx%self.cats + 1)

        # choosing our pair of positive images (im1, im2)
        positives = os.listdir(os.path.join(self.path, idx))
        im1, im2 = random.sample(positives, 2)

        # choosing a negative class and negative image (im3)
        negative_cats = [str(x+1) for x in range(self.cats)]
        negative_cats.remove(idx)
        negative_cat = str(random.choice(negative_cats))
        negatives = os.listdir(os.path.join(self.path, negative_cat))
        im3 = random.choice(negatives)

        im1,im2,im3 = os.path.join(self.path, idx, im1), os.path.join(self.path, idx, im2), os.path.join(self.path, negative_cat, im3)
        im1 = Image.open(im1).convert("RGB")
        im2=Image.open(im2).convert("RGB")
        im3=Image.open(im3).convert("RGB")
        im1 = self.transforms(im1)
        im2 = self.transforms(im2)
        im3 = self.transforms(im3)

        return [im1, im2, im3]

    # we'll put some value that we want since there can be far too many triplets possible
    # multiples of the number of images/ number of categories is a good choice
    def __len__(self):
        return self.cats*8


# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
PATH_TRAIN="/content/drive/MyDrive/IRMA/datatrainsplit"
train_data = TripletData(PATH_TRAIN, train_transforms,"train")
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=35, shuffle=True, num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)

    # Distances in embedding space is calculated in euclidean
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [ ]:
epochs =6
device = 'cuda'
# Our base model
model = models.resnet18().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
triplet_loss = TripletLoss()
max_loss=1000
# Training
for epoch in range(epochs):

    model.train()
    epoch_loss = 0.0
    for data in tqdm(train_loader):
        optimizer.zero_grad()
        x1,x2,x3 = data
        e1 = model(x1.to(device))
        e2 = model(x2.to(device))
        e3 = model(x3.to(device))


        loss = triplet_loss(e1,e2,e3)
        epoch_loss += loss
        loss.backward()
        optimizer.step()


    print("Train Loss: {}".format(epoch_loss.item()))
    print("max loss",max_loss)
    if epoch_loss.item() < max_loss:
     print("model saved")
     max_loss=epoch_loss.item()
     torch.save(model.state_dict(), "_resnet18_minSGD.pt")
    torch.save(model.state_dict(), "_resnet18_SGD.pt")

  0%|          | 0/14 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Loss: 74.24512481689453
max loss 1000
model saved


  0%|          | 0/14 [00:00<?, ?it/s]

Train Loss: 22.809131622314453
max loss 74.24512481689453
model saved


...

Train Loss: 18.26801109313965
max loss 18.521032333374023
model saved


  0%|          | 0/14 [00:00<?, ?it/s]

Train Loss: 15.73271656036377
max loss 18.26801109313965
model saved


  0%|          | 0/14 [00:00<?, ?it/s]

Train Loss: 16.767545700073242
max loss 15.73271656036377


In [ ]:
device = 'cuda'
ENCODER_MODEL_PATH ="/content/drive/MyDrive/sgdresnet/_resnet18_SGD.pt"
model = models.resnet18().cuda()
model.load_state_dict(torch.load(ENCODER_MODEL_PATH, map_location=device))

<All keys matched successfully>

In [ ]:
!pip install faiss-gpu
#!pip install faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
co=0
faiss_index = faiss.IndexFlat(1000,faiss.met)    # build the index
model.eval()
im_indices = []
with torch.no_grad():
    for f in glob.glob(os.path.join(PATH_TRAIN, '*/*')):
        co=co+1
        print("image", co)
        im = Image.open(f).convert("RGB")
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()
        preds = model(im)
        preds = np.array([preds[0].cpu().numpy()])
        faiss_index.add(preds) #add the representation to index
        im_indices.append(f)
#faiss.write_index(faiss_index, "/content/drive/MyDrive/sgdresnet/index1.index")
#df = pd.DataFrame(im_indices)
#df.to_csv('/content/drive/MyDrive/color/coul/static/modFeat/2/indicesirma.csv')

In [ ]:
index8 = faiss.read_index("/content/drive/MyDrive/IRMA/index1.index",faiss.FlatData)

index4=faiss.FileIOReader("/content/drive/MyDrive/color/coul/static/modFeat/2/index.index")
df=pd.read_csv('/content/drive/MyDrive/color/coul/static/modFeat/2/indicesirma.csv')
print(df.iloc[1][1])

In [ ]:
import pandas as pd
csvtrain=pd.read_csv("/content/drive/MyDrive/IRMA/ImageCLEFmed2009_train_codes.02.csv",sep=';')
csvtrain=csvtrain.set_index("image_id",drop=False)
csvtest=pd.read_csv("/content/drive/MyDrive/IRMA/ImageCLEFmed2009_test_codes.03.csv",sep=';')
csvtest=csvtest.set_index("image_id",drop=False)

In [ ]:
def Averageprecision(code, classe,indices):
            correctImages = []
            SimImg=0
            compRes=0
            for index in indices:
              idimage=os.path.splitext(os.path.basename(im_indices[index]))[0]

              classesimilaire=csvtrain.at[int(idimage),"05_class"]
              codesimil=csvtrain.at[int(idimage),"irma_code"]
              codesimil=codesimil.split('-')
              codesimil=codesimil[2]

              print("la classe IMG req",classe ,"les code",code,"classe similaire or not est", classesimilaire,"le code",codesimil)
              #print("la classe 6 de limage similaire or not est", classesimilaire1)
              compRes=compRes+1
              if classe==classesimilaire or code==codesimil :
                 SimImg=SimImg+1
                 num =(SimImg)/compRes
                 correctImages.append(num)
            if SimImg==0:
                 SimImg=1
            return sum(correctImages)/SimImg

In [ ]:
from faiss.swigfaiss import GpuIndex
def meanAveragePrecision1(maxResults):
   map = 0
   number=0
   c=0
   path ="/content/drive/MyDrive/IRMA/Vidange/"
   #path ="/content/drive/MyDrive/IRMA/test/"
   #os.mkdir('/content/drive/MyDrive/IRMA/Vidange')
   with torch.no_grad():
    for f in os.listdir(path):
     number=number+1
     print("path of the test photo", number ," ",f)
     im = Image.open(os.path.join(path,f)).convert("RGB")
     im = im.resize((224,224))
     im = torch.tensor([val_transforms(im).numpy()]).cuda()
     test_embed = model(im).cpu().numpy()

     _, I =faiss_index.search(test_embed, 10)
     indices_list = I.tolist()
     indices = indices_list[0]
     print(indices)
     idimage=os.path.splitext(f)[0]
     idimage=re.sub(r'\(.+\)','',idimage)
     print("the pzth",f,"idimage",idimage)
     classe=csvtest.at[int(idimage),"05_class"]
     code=csvtest.at[int(idimage),"irma_code"]
     code=code.split('-')
     code=code[2]
     ap = Averageprecision(code,classe,indices)
     print("ap",ap)
     map = map + ap
     print("map is: ",map)

   return map/1733

In [ ]:
print("maaap:",meanAveragePrecision1(10))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
la classe IMG req 40 les code 913 classe similaire or not est 11 le code 310
la classe IMG req 40 les code 913 classe similaire or not est 18 le code 922
ap 0.16666666666666666
map is:  538.9835577286475
path of the test photo 1401   15513.png
[5692, 9650, 9614, 5015, 4541, 3940, 7842, 5672, 3613, 9682]
the pzth 15513.png idimage 15513
la classe IMG req 6 les code 914 classe similaire or not est 32 le code 463
la classe IMG req 6 les code 914 classe similaire or not est 29 le code 414
la classe IMG req 6 les code 914 classe similaire or not est 29 le code 414
la classe IMG req 6 les code 914 classe similaire or not est 7 le code 330
la classe IMG req 6 les code 914 classe similaire or not est 6 le code 915
la classe IMG req 6 les code 914 classe similaire or not est 3 le code 415
la classe IMG req 6 les code 914 classe similaire or not est 4 le code 200
la classe IMG req 6 les code 914 classe similaire or not